Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.linear_model import LogisticRegression

import cv2
from google.colab.patches import cv2_imshow
import time

Loading the beans dataset into train and test

In [ ]:
image_train, label_train = tfds.as_numpy(tfds.load(
    'beans',
    split='train',
    batch_size=-1,
    as_supervised=True,
))

print("Train data info:",type(image_train), image_train.shape)

image_test, label_test = tfds.as_numpy(tfds.load(
    'beans',
    split='test',
    batch_size=-1,
    as_supervised=True,
))

print("Test data info:",type(image_test), image_test.shape)

Preprocessing the train and test data

In [ ]:
x_train = []
x_train_normalized = []

for i in image_train:
  x_train.append(i.astype("float32"))
for i in x_train:
  x_train_normalized.append(i/np.max(i))

print(x_train[0].shape, x_train_normalized[0].shape)

x_train_resized = []
for i in x_train_normalized:
  im = cv2.resize(i, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
  x_train_resized.append(im)

# grayscale pixel as features for training the logistic model 
# x_train_grey = []
# for i in x_train_resized:
#   grayscale = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
#   x_train_grey.append(grayscale) #np.dot(i,[0.2989,0.5870,0.1140])

# print(x_train_resized[0].shape, x_train_grey[0].shape)

x_test = []
x_test_normalized = []

for i in image_test:
  x_test.append(i.astype("float32"))
for i in x_test:
  x_test_normalized.append(i/np.max(i))

x_test_resized = []
for i in x_test_normalized:
  im = cv2.resize(i, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
  x_test_resized.append(im)

# x_test_grey = []
# for i in x_test_resized:
#   grayscale = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
#   x_test_grey.append(grayscale) #np.dot(i,[0.2989,0.5870,0.1140])
#   # x_test_grey.append(np.dot(i,[0.2989,0.5870,0.1140]))